<a href="https://colab.research.google.com/github/jkimbf/FL-Tutorials/blob/master/PySyft_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySyft
**PySyft** is a wrapper around PyTorch and adds extra functionality to it to facilitate federated learning.

<br> Installation: ```pip install syft``` ([Installation guide](https://github.com/OpenMined/PySyft/blob/master/INSTALLATION.md))

<br> How do we train a model on data we don't have access to or cannot see?
> We want to perform deep learning on some **remote machines**. <br> Thus, instead of using Torch ```tensor```, we are now going to work with a **```pointer``` to ```tensor```** <br> (this ```tensor``` is stored on the remote location) with PySyft.

---










## I. Imports and Hook

```
import torch
import syft as sy
hook = sy.TorchHook(torch)
```
```TorchHook```: wrapping by "adding all the additional functionality" to Torch for Federated Learning and other Private AI techniques.

<br> **Hook** overrides mothods on PyTorch Tensors.
> to extend torch methods to allow for moving of tensors from one worker to another <br> to execute commands on one worker that are called on tensors controlled by the local worker

<br> **FIRST THING** I will initialize when using PySyft with PyTorch <br> --> **Augmenting PyTorch** with PySyft's added functionality (e.g. remote execution)



---



## II. Creating a Virtual Worker

```
# Create a Machine owned by harmony clinic
harmony_clinic = sy.VirtualWorker(hook=hook, id = 'clinic')
```
What does it mean by a ```pointer``` to ```tensor```
> Let's create a virtual machine first (e.g. owned by some health clinic, say ```harmony_clinic```) <br> Use this to simulate a machine present at a remote location <br>Note: ```syft``` calls these machines as ```VirtualWorker```


---




## III. Sending the Tensors
Now I know that the ```harmony_clinic``` is at a random location.

Now, let's create some data and send it to ```harmony_clinic```
```
# Create a Tensor (e.g. some gene sequence)
dna = torch.tensor([0,1,2,1,2])

# Send it, and in turn get a pointer back that points to that Tensor
dna_ptr = dna.send(harmony_clinic)

print(dna_ptr)
```
```PointerTensor``` points from ```me``` to ```harmony_clinic``` (random numbers are the object IDs assigned by PySyft)

<br> Now, ```harmony_clinic``` has the **tensor**.

```harmony_clinic._objects``` shows the objects that ```harmony_clinic``` currently has
<br>
```
print(harmony_clinic._objects)
```

---




## IV. Getting back the Tensors
```.get()```: **take the ```tensor``` back** from a remote location
```
# get back dna
dna = dna_ptr.get()
print(dna)    # tensor({0, 1, 2, 1, 2})

# clinic no longer has tensor dna, moved back to out machine
print(harmony_clinic._objects())    # {}
```

I can ```use pointers to do arithmetic``` as in PyTorch


---



## V. Doing Deep Learning with Pointer Tensors
```
a = torch.tensor([3.14, 6.28]).send(harmonic_clinic)
b = torch.tensor([6.14, 3.28]).send(harmonic_clinic)
c = a + b
print(c)
```
When ```c = a + b```, a command is sent to the **"""remote machine"""** to **"""do exact calculation"""**
> Creates a ```new tensor``` on its machine <br> send back a ```pointer to tensor c```

This API had been extended to all the PyTorch operations including Back propagation

Meaning that, 
<br>```We can use the same PyTorch code that we usually do when doing Maching Learning```

```
# Create two tensors and send it
train = torch.tensor([2.4, 6.2], requires_grad=True).send(harmony_clinic)
label = torch.tensor([2, 6.]).send(harmony_clinic)

# Apply some function
# Since this is just to show the idea, use rather simple one, L1 loss
loss = (train-label).abs().sum()

# Even .backward() works with pointers
loss.backward()

# Retrieve back the train tensor
train = train.get()

print(train)    # tensor([2.4, 6.2], requires_grad=True)

# If everything went well, "graditents accumulated" should be in .grad attribute of train
print(train.grad)    # tensor([1., 1.])
```

the API is quite flexible and capable of performing nearly any operation I would normally perform in Torch on remote data

---

